In [1]:
# Author: Robert Guthrie

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)
import utils

In [2]:
import nltk
from sklearn.metrics import classification_report, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
import torch
import torch.nn as nn

#from torch_model_base import TorchModelBase
#from torch_shallow_neural_classifier import TorchShallowNeuralClassifier
from torch_rnn_classifier import TorchRNNDataset, TorchRNNClassifier, TorchRNNModel
import utils

import pandas as pd
from collections import Counter

In [3]:
def prepare_sequence(seq, to_ix): # converts one list of tokens into one tensor of token ids
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    # Inp: torch array (1,nTags) of next tag scores (accumul transmission + emission); Out: single number tensor
    # see crfCalcs.xlsx
   # max_score = vec[0, argmax(vec)]
    max_score = vec[0, torch.argmax(vec,axis=1)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

def prepare_sequences(seqs, to_ix):
    idxs = [prepare_sequence(seq, to_ix) for seq in seqs]
    #return torch.tensor(idxs, dtype=torch.long)
    return idxs

In [456]:
from torch import manual_seed
torch.manual_seed(0)
a = torch.randn((1,5))
print(a)
b = log_sum_exp(a)
print(b)

tensor([[ 1.5410, -0.2934, -2.1788,  0.5684, -1.0845]])
tensor([2.0323])


In [463]:
c = torch.randn((2,5))
print(c)
for i, feat in enumerate(c):
    print(i)
    print(feat)

tensor([[ 0.2514, -1.8709, -0.0724,  0.1736, -2.0898],
        [-0.2333, -0.3052, -0.4891, -1.5174, -0.6644]])
0
tensor([ 0.2514, -1.8709, -0.0724,  0.1736, -2.0898])
1
tensor([-0.2333, -0.3052, -0.4891, -1.5174, -0.6644])


In [4]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix 
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)
       
        self.transitions = self.init_transitions()

        self.hidden = self.init_hidden()

    def init_transitions(self): # Matrix of transition parameters. Entry i,j is the score of transitioning *to* i *from* j.
        # KEEP
        transitions = nn.Parameter(torch.randn(self.tagset_size, self.tagset_size)) # initialize w/ random numbers (5,5) array
        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        transitions.data[tag_to_ix[START_TAG], :] = -10000
        transitions.data[:, tag_to_ix[STOP_TAG]] = -10000   
        return transitions

    def init_hidden(self):
        # KEEP
        return (torch.randn(2, 1, self.hidden_dim // 2), # here are dividing hidden dim by 2; since bidir end up w/ 2x (=4 tot hid layer)
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # KEEP
        # Inp: feats is tensor (noTokens,noDiffLabelClasses = tagset_size) = (11,5); Out: tensor 1d
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.) # Inp: self.tagset_size=5 ; Out: tensor (1,5)=-10000
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0. # updates START_TAG Id of init_alphas[0] to be = 0

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats: # iterate through each diff token (feat)
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size): # iterate through each tag
                # broadcast the emission score: it is the same regardless of the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size) # feat[next_tag] is tensor sngl no. w/ gradFn="select_bckwd"; 
                # emit score is (1,tagset_size) w/ gradFn="ExpandBackward"
                # note: in this formula view(1,-1) squishes sngl no. into (1,1) array and expand(1,5) expands it into (1,5) array
                ###########################################################################################
                # the ith entry of trans_score is the score of transitioning to next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1) # self.transitions is (5,5) torch array; trans_score is (1,5)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score # next_tag_var is (1,5)
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        # Input: sentence is list (noTokens); Output lstm_feats is tensor (noTokens,noDiffLabelClasses = len(tag_to_ix))
        self.hidden = self.init_hidden() # self.hidden becomes (a,b) where both are tensors (2,1,2) hidden_dim=4 // 2
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1) # embeds is tensor (noTokens,1,embedding_dim) = (11,1,6)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden) # self.hidden becomes (a,b) where both are tensors (2,1,2) hidden_dim=4 // 2
        # lstm_out is (11,1,4) - last dim is = hidd_dim*2 as bidirect LSTM (NOTE: hidd_dim = self.hidden_dim / 2)
        # Note also: by default there is NO BATCHING so lstm_out is (seq, batch, feature) instead of (batch, seq, feature)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim) # "squishes" middle redundant dimension: (11,4)
        lstm_feats = self.hidden2tag(lstm_out) # FCL that maps from (11,4) to (11,5) where 5 is the tagset space
       # print("lstm_feats")
       # print(lstm_feats.shape)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # KEEP
        # Gives the score of a provided tag sequence - see crfCalcs.xlsx
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags]) # add START_TAG to sentence tags
        for i, feat in enumerate(feats): # iterating thru' all tokens of feats; columns are 1 ... nTags
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats): # feats is (11,5) i.e. LSTM produced prob of each token being associated w/ each of the 5 tags
        # KEEP
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                # best_tag_id = argmax(next_tag_var)
                best_tag_id = torch.argmax(next_tag_var,axis=1)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        # best_tag_id = argmax(terminal_var)
        best_tag_id = torch.argmax(terminal_var,axis=1)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        # KEEP
        # Inputs: sent_in is list (noTokens); targets is torchArray (noTokens); Ouput: loss is single number
        feats = self._get_lstm_features(sentence) # feats is (noTokens,noDiffLabelClasses)
        forward_score = self._forward_alg(feats) # fwd score from model (accum. transmissions + emissions (i.e. feats))
        gold_score = self._score_sentence(feats, tags) # gold score from self.transitions, feats (emissions), seq of tags in example sentence
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # KEEP
        # Inp: sentence (1 example) as list of tokenIds; Out: score (1d), tag_seq (noTokens) with forecasted tagId for each token
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence) # (noTokens,noDiffLabelClasses)
       # print(lstm_feats)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        print(tag_seq)
        return score, tag_seq

In [5]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 6
HIDDEN_DIM = 4
#tol = 1e-05
#n_iter_no_change = 10
#validation_fraction=0.1

# Make up some training data
training_data = [(
    "the wall street journal reported today that apple corporation made money".split(),
    "B I I I O O O B I O O".split()
), (
    "georgia tech is a university in georgia".split(),
    "B I O O O O B".split()
)]

## 5 lines below create dict of (wordToken,wordId)
word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix) # i.e. each successive word added gets a successive index

tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}

vocab = list(set([item for auxList,_ in training_data for item in auxList])) + ["$UNK"]

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
#model = TorchLSTM_CRFSequenceLabeler(
#    len(word_to_ix), 
#    tag_to_ix, 
#    EMBEDDING_DIM, 
#    HIDDEN_DIM,
#    vocab,
#    early_stopping=True,
#    eta=0.001)

optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)
#print(model.parameters().__next__())

# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
    # print(model(precheck_sent))

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(1):
        #300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix) # sentence_in is single example (i.e. sentence) with tensor of tokens converted to ids
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long) # convert labels classes to ids

        # Step 3. Run our forward pass.
        #print("sent_in length"+str(len(sentence_in)))
        #print(targets.shape)
        #print(sentence_in)
        loss = model.neg_log_likelihood(sentence_in, targets) # sent_in is list (noTokens); targets is torchArray (noTokens); # loss is single number
        #print("loss length"+str(len(loss))) 

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()
    #print(epoch)

# Check predictions after training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    # print(model(precheck_sent))

In [6]:
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[1][0], word_to_ix) # convert o tokenIds
   # print(precheck_sent)
    print(model.forward(precheck_sent))

[tensor([1]), tensor([1]), tensor([1]), tensor([1]), tensor([1]), tensor([1]), tensor([1])]
(tensor([3.7316]), [tensor([1]), tensor([1]), tensor([1]), tensor([1]), tensor([1]), tensor([1]), tensor([1])])


In [430]:
a=torch.randn(1)
print(a.shape)
b=a.view(1,-1).expand(1,5)
print(b.shape)

torch.Size([1])
torch.Size([1, 5])


In [57]:
print(training_data)
X_train = [[item for item in auxList] for auxList,_ in training_data]
y_train = [[item for item in auxList] for _,auxList in training_data]
print(len(X_train))
print(len(y_train))
print(vocab)

[(['the', 'wall', 'street', 'journal', 'reported', 'today', 'that', 'apple', 'corporation', 'made', 'money'], ['B', 'I', 'I', 'I', 'O', 'O', 'O', 'B', 'I', 'O', 'O']), (['georgia', 'tech', 'is', 'a', 'university', 'in', 'georgia'], ['B', 'I', 'O', 'O', 'O', 'O', 'B'])]
2
2
['money', 'is', 'a', 'street', 'reported', 'apple', 'university', 'wall', 'tech', 'made', 'that', 'journal', 'the', 'today', 'georgia', 'in', 'corporation', '$UNK']


In [433]:
# reload vsm module
import torch_rnn_classifier, torch_model_base
import importlib
importlib.reload(torch_rnn_classifier)
importlib.reload(torch_model_base)

<module 'torch_model_base' from 'C:\\Users\\vasco\\DeepLearning-JN\\cs224u\\base_data\\Vasco\\torch_model_base.py'>

In [350]:
seq_mod = TorchLSTM_CRFSequenceLabeler(
    vocab,
    early_stopping=True,
    eta=0.001)

    #len(word_to_ix), 
    #tag_to_ix, 
    #EMBEDDING_DIM, 
    #HIDDEN_DIM,

In [355]:
print(X_train[0])
print(y_train[0])

['the', 'wall', 'street', 'journal', 'reported', 'today', 'that', 'apple', 'corporation', 'made', 'money']
['B', 'I', 'I', 'I', 'O', 'O', 'O', 'B', 'I', 'O', 'O']


In [351]:
%time _ = seq_mod.fit(X_train, y_train)

tensor([7], device='cuda:0')
torch.Size([1, 7])


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)

In [380]:
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    print(precheck_sent)
    print(model.forward(precheck_sent))

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 13, 25, 21, 26,  2, 11, 27, 28, 29, 30, 31,
        32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 32, 44, 45, 37, 46, 47,
        48, 49, 50, 51, 17, 21, 52, 53,  2, 54, 55, 56, 57, 21, 58, 59, 60, 27,
        61, 62, 21, 21, 21])
(tensor([545.6737]), [tensor([0]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([0]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([11]), tensor([2]), tensor([11]), tensor([11]), tensor([11]), te

In [101]:
print(training_data[0][0])

['DORNBIRN', 'In', 'der', 'Schulgasse', 'in', 'Dornbirn', 'hat', 'eine', '71,93', 'Quadratmeter', 'große', 'Wohnung', 'für', 'einen', 'Quadratmeterpreis', 'von', '5533,71', 'Euro', 'den', 'Besitzer', 'gewechselt', '.', 'Dieser', 'beinhaltet', 'auch', 'einen', 'Pkw-Abstellplatz', '.', 'Käufer', 'der', 'Wohnung', 'mit', '9,86', 'Quadratmetern', 'Terrasse', 'ist', 'die', 'ValLiLean', 'Beteiligungs-', 'und', 'Immobilienverwaltungs', 'GmbH', 'Beim', 'Verkäufer', 'handelt', 'es', 'sich', 'um', 'die', 'Karrenblick', 'Projekt', 'GmbH', ' ', 'Der', 'Kaufpreis', 'liegt', 'bei', '398.040', 'Euro', '.', 'Unterzeichnet', 'wurde', 'der', 'Kaufvertrag', 'am', '18.', 'September', '.', 'Die', 'Verbücherung', 'datiert', 'mit', 'Oktober', '2020', '.', '.', '.']


In [11]:
print(word_to_ix)

{'the': 0, 'wall': 1, 'street': 2, 'journal': 3, 'reported': 4, 'today': 5, 'that': 6, 'apple': 7, 'corporation': 8, 'made': 9, 'money': 10, 'georgia': 11, 'tech': 12, 'is': 13, 'a': 14, 'university': 15, 'in': 16}


In [37]:
print(training_data)

[(['the', 'wall', 'street', 'journal', 'reported', 'today', 'that', 'apple', 'corporation', 'made', 'money'], ['B', 'I', 'I', 'I', 'O', 'O', 'O', 'B', 'I', 'O', 'O']), (['georgia', 'tech', 'is', 'a', 'university', 'in', 'georgia'], ['B', 'I', 'O', 'O', 'O', 'O', 'B'])]


Run with our data

In [18]:
import json

In [325]:
### RUN 
### NOTE: Make sure to copy most up-to-date annotations2.jsonl file to /Vasco/
with open('annotations2.jsonl') as jsonl_file:
    # note: after running data-preprocessing.ipynb this file already has token-level labels
    lines = jsonl_file.readlines()
annot = [json.loads(line) for line in lines]
#print(annot[0])

In [327]:
### RUN
# now convert annotation tokens into list (sentences) of lists (tokens) format for sklearn_crfsuite.CRF
train_sents=[] 
for j in range(0,len(annot)):
    a = annot[j]['tokens']
    # Only add sample if there are annotations
    if annot[j]['spans']!=[]:
        train_sentence = []
        for i in range(0,len(a)):
            if 'label' in a[i]: # only add element if this sample sentence has been labelled 
                token_element = (a[i]['text'],a[i]['label'])
                train_sentence.append(token_element)
        train_sents.append(train_sentence)

In [322]:
print(train_sents[0])

[('DORNBIRN', 'ORT'), ('In', 'O'), ('der', 'O'), ('Schulgasse', 'STRASSE'), ('in', 'O'), ('Dornbirn', 'ORT'), ('hat', 'O'), ('eine', 'O'), ('71,93', 'FLAECHE'), ('Quadratmeter', 'O'), ('große', 'O'), ('Wohnung', 'IMMO_TYP'), ('für', 'O'), ('einen', 'O'), ('Quadratmeterpreis', 'O'), ('von', 'O'), ('5533,71', 'QMPREIS'), ('Euro', 'O'), ('den', 'O'), ('Besitzer', 'O'), ('gewechselt', 'O'), ('.', 'O'), ('Dieser', 'O'), ('beinhaltet', 'O'), ('auch', 'O'), ('einen', 'O'), ('Pkw-Abstellplatz', 'O'), ('.', 'O'), ('Käufer', 'O'), ('der', 'O'), ('Wohnung', 'O'), ('mit', 'O'), ('9,86', 'TERRASSENGROESSE'), ('Quadratmetern', 'O'), ('Terrasse', 'O'), ('ist', 'O'), ('die', 'O'), ('ValLiLean', 'KAEUFER'), ('Beteiligungs-', 'KAEUFER'), ('und', 'KAEUFER'), ('Immobilienverwaltungs', 'KAEUFER'), ('GmbH', 'KAEUFER'), ('Beim', 'O'), ('Verkäufer', 'O'), ('handelt', 'O'), ('es', 'O'), ('sich', 'O'), ('um', 'O'), ('die', 'O'), ('Karrenblick', 'VERKAEUFER'), ('Projekt', 'VERKAEUFER'), ('GmbH', 'VERKAEUFER'), (

In [149]:
# put into format expected above
all_data = [([a1 for a1,a2 in el],[a2 for a1,a2 in el]) for el in train_sents]
#tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}
tag_to_ix = {'ORT': 0, 'STRASSE': 1, 'FLAECHE': 2, 'IMMO_TYP': 3, 'QMPREIS': 4, 'TERRASSENGROESSE': 5, 'KAEUFER': 6, 'VERKAEUFER': 7, 'GESAMTPREIS': 8, 'DATUM_VERTRAG': 9, 'DATUM_VERBUECHERUNG': 10, 'O': 11, START_TAG: 12, STOP_TAG: 13}

In [51]:
# check
for i in range(0,len(train_sents)):
    if len(train_sents[i])==0:
        print(i)

In [97]:
print(len(training_data))

133


In [150]:
word_to_ix = {}
for sentence, tags in all_data: # i.e. before splitting training / test data
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix) # i.e. each successive word added gets a successive index

In [152]:
# split training / test.
training_data=all_data[:100]
test_data=all_data[100:]

In [184]:
# generate predictions v2

sentences = [a for a,_ in test_data]
labels = [b for _,b in test_data]
print(sentences[0])
print(prepare_sequence(sentences[0], word_to_ix))
#test_data_preds = model(prepare_sequence(sentences[0], word_to_ix))
#print(test_data_preds)
print(labels[0])
test_data_preds = [model(prepare_sequence(a, word_to_ix)) for a in sentences]
print(test_data_preds[0])
test_data_gold = [[tag_to_ix[el] for el in a] for a in labels] 

['Nenzing', ':', '50', 'ha']
tensor([882, 385, 883, 878])
['ORT', 'O', 'O', 'O']
(tensor([14.7289], grad_fn=<IndexBackward>), [tensor([0]), tensor([11]), tensor([11]), tensor([11])])


In [223]:
# now unfold both gold and preds list of tensors - not necessary for sklearn classification_report
#test_data_preds_unfold = [int(item) for sublist in test_data_preds for item in sublist[1]]
#test_data_gold_unfold = [int(item) for sublist in test_data_gold for item in sublist]

In [253]:
ix_to_tag = {value: key for key, value in tag_to_ix.items()}
print(ix_to_tag)

{0: 'ORT', 1: 'STRASSE', 2: 'FLAECHE', 3: 'IMMO_TYP', 4: 'QMPREIS', 5: 'TERRASSENGROESSE', 6: 'KAEUFER', 7: 'VERKAEUFER', 8: 'GESAMTPREIS', 9: 'DATUM_VERTRAG', 10: 'DATUM_VERBUECHERUNG', 11: 'O', 12: '<START>', 13: '<STOP>'}


In [254]:
#test_data_preds_unfold_tag = [str(ix_to_tag[el]) for el in test_data_preds_unfold]
#test_data_gold_unfold_tag = [str(ix_to_tag[el]) for el in test_data_gold_unfold]

In [229]:
labels_distinct = list(tag_to_ix.keys())

In [248]:
print(sorted_labels)

['O', 'KAEUFER', 'DATUM_VERBUECHERUNG', 'DATUM_VERTRAG', 'VERKAEUFER', 'TERRASSENGROESSE', 'GESAMTPREIS', 'FLAECHE', 'IMMO_TYP', 'QMPREIS', 'ORT', '<START>', '<STOP>', 'STRASSE']


In [211]:
print(list(test_data_preds[0][1]))

[tensor([0]), tensor([11]), tensor([11]), tensor([11])]


In [191]:
from sklearn_crfsuite import metrics

In [272]:
# don't unfold but convert ids to tags:
test_data_preds_tag = [[str(ix_to_tag[int(item)]) for item in sublist[1]] for sublist in test_data_preds]
test_data_gold_tag = [[str(ix_to_tag[int(item)]) for item in sublist] for sublist in test_data_gold ]

In [279]:
sorted_labels = sorted( # labels are coming in as id's not names
    labels_distinct,
    key=lambda name: (name[1:], name[0])
)

print(metrics.flat_classification_report(
    test_data_gold_tag, test_data_preds_tag, labels=labels_distinct, digits=3
))

                     precision    recall  f1-score   support

                ORT      0.730     0.540     0.621        50
            STRASSE      0.000     0.000     0.000        19
            FLAECHE      0.000     0.000     0.000        17
           IMMO_TYP      0.000     0.000     0.000        22
            QMPREIS      0.000     0.000     0.000        11
   TERRASSENGROESSE      0.000     0.000     0.000         5
            KAEUFER      0.000     0.000     0.000        22
         VERKAEUFER      0.500     0.433     0.464        30
        GESAMTPREIS      0.000     0.000     0.000        22
      DATUM_VERTRAG      0.632     0.774     0.696        31
DATUM_VERBUECHERUNG      0.913     0.724     0.808        29
                  O      0.878     0.974     0.923      1107
            <START>      0.000     0.000     0.000         0
             <STOP>      0.000     0.000     0.000         0

          micro avg      0.852     0.852     0.852      1365
          macro avg   

C:\Users\vasco\anaconda3\envs\xcs224u\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vasco\anaconda3\envs\xcs224u\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [340]:
all_words = [el for auxList1,_ in all_data for el in auxList1]
vocab = list(set(all_words)) + ["$UNK"]

In [135]:
a= torch.randn(2,3)
c=torch.randn(2)
print(a)
print(c)
for b in range(0,len(a)):
    print(a[b])
    print(c[b])


tensor([[-0.4675,  0.8189, -0.4107],
        [ 0.3148, -0.9794,  0.1212]])
tensor([-0.1895,  1.6988])
tensor([-0.4675,  0.8189, -0.4107])
tensor(-0.1895)
tensor([ 0.3148, -0.9794,  0.1212])
tensor(1.6988)


In [383]:
seq_mod = TorchLSTM_CRFSequenceLabeler(
    vocab,
    early_stopping=True,
    eta=0.001)

In [384]:
%time _ = seq_mod.fit(X_train, y_train)

Stopping after epoch 19. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 2.246312379837036

Wall time: 3.44 s


In [385]:
y_pred = seq_mod.predict(X_test)
print(y_test[0])
print(y_pred[0])

['ORT', 'O', 'O', 'STRASSE', 'O', 'ORT', 'O', 'O', 'IMMO_TYP', 'O', 'O', 'IMMO_TYP', 'O', 'FLAECHE', 'O', 'O', 'GESAMTPREIS', 'O', 'O', 'O', 'O', 'O', 'IMMO_TYP', 'O', 'FLAECHE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'VERKAEUFER', 'O', 'O', 'O', 'O', 'O', 'O', 'DATUM_VERTRAG', 'DATUM_VERTRAG', 'DATUM_VERTRAG', 'O', 'O', 'O', 'O', 'O', 'DATUM_VERBUECHERUNG', 'DATUM_VERBUECHERUNG', 'O']
['VERKAEUFER', 'VERKAEUFER', 'O', 'O', 'O', 'STRASSE', 'O', 'O', 'QMPREIS', 'O', 'QMPREIS', 'QMPREIS', 'STRASSE', 'STRASSE', 'STRASSE', 'STRASSE', 'DATUM_VERBUECHERUNG', 'O', 'QMPREIS', 'O', 'O', 'O', 'O', 'O', 'VERKAEUFER', 'O', 'O', 'O', 'O', 'O', 'DATUM_VERBUECHERUNG', 'DATUM_VERBUECHERUNG', 'O', 'KAEUFER', 'O', 'QMPREIS', 'QMPREIS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'STRASSE', 'O', 'DATUM_VERBUECHERUNG', 'DATUM_VERBUECHERUNG', 'O']


In [309]:
EMBEDDING_DIM = 5
HIDDEN_DIM = 4

In [376]:
seq_mod = BiLSTM_CRF_2(
    vocab,
    tag_to_ix,
    EMBEDDING_DIM,
    HIDDEN_DIM,
    early_stopping=True,
    eta=0.001)

In [328]:
X_all, y_all = [[a for a,_ in el] for el in train_sents ], [[b for _,b in el] for el in train_sents]

In [336]:
# VR ADDED March 30th
from sklearn.utils import shuffle

train_ratio = 0.75
train_test_split = round(0.75*len(X_all) - 0.5) # -0.5 => floor
idx = [i for i in range(0,len(X_all))]
idx_shuffle = shuffle(idx,random_state=0)
X_shuffle, y_shuffle = [X_all[auxIdx] for auxIdx in idx_shuffle], [y_all[auxIdx] for auxIdx in idx_shuffle]

X_train, X_test, y_train, y_test = X_shuffle[:train_test_split], X_shuffle[train_test_split:], y_shuffle[:train_test_split], y_shuffle[train_test_split:]

In [354]:
#print(vocab['$UNK'])
index = dict(zip(vocab, range(len(vocab))))
print(index['$UNK'])

1107


In [361]:
print(y_train[0])

['ORT', 'O', 'O', 'O']


In [360]:
a=sorted(set(y_train))

TypeError: unhashable type: 'list'

In [377]:
%time _ = seq_mod.fit(X_train, y_train)

AttributeError: 'int' object has no attribute 'embedding_dim'